# Out of core computing

In this notebook we will demonstrate how to use `dask` to perform computation on a file of 150GB that cannot be fully loaded in memory (a Comet computing node has 128 GB of RAM)

In [ ]:
!ls -lah /oasis/scratch/comet/zonca/temp_project/random_150GB.h5

## Create a h5py dataset

A `h5py` dataset references the data on disk without loading them to memory unless explicitely asked for.

In [ ]:
import h5py
import os
f = h5py.File(os.path.join('..', 'data', 'random.hdf5'))
dset = f['/x']

In [ ]:
dset.shape[0] / 1e9

In [ ]:
dset.dtype

5 billion `float32` floating point numbers

## Compute the sum with `dask`

We can create a `dask` array from any object that presents the same interface as `numpy` arrays, in this case a `h5py` dataset. The chunk size is defining how big is each subsection of the array that is going to be loaded and manipulated by `dask`, many chunks can be loaded simultaneously in memory to make use of multiple cores.

In [ ]:
import dask.array as da
chunk_size = int(1e7)
x = da.from_array(dset, chunks=(chunk_size,))

In [ ]:
print("Number of chunks", x.numblocks)

`dask` computations are **lazy**, they are not evaluated immediately because `dask` can combine different operations together and optimize it computation.

In [ ]:
result = x.mean()
result

`num_workers` specifies the number of threads to be used, in this case we are bound by loading data from disk.

In [ ]:
%time result.compute(num_workers=24)

You can play with the `chunk_size` and `num_workers` to have a feeling of how performance scales.